<a href="https://colab.research.google.com/github/AtillaMehmet/FinalProjectPrototype/blob/main/Chatbot_prompts_working_mostly_still_not_good_accuracy_but_to_be_expected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
import string
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
# Load JSON data
data_file = open('/content/intents.json').read()
intents_data = json.loads(data_file)

# Access the 'intents' list under the 'root' key
intents = intents_data['root']['intents']

# Now we can iterate over the intents
for intent in intents:
    tag = intent['tag']
    patterns = intent['patterns']
    responses = intent['responses']
    context_set = intent['context_set']

In [ ]:
documents = []  # Initialize the documents list
words = []
classes = []

for intent in intents:
    for pattern in intent['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)  # add each element into list
        # combination between patterns and intents
        documents.append((w, intent['tag']))  # add single element to end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



In [ ]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
ignore_words = ['?', '!', ',', '.']  # Define the words you want to ignore

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

70 documents
 [(['hello'], 'greeting'), (['hi'], 'greeting'), (['hey', 'you'], 'greeting'), (['yo'], 'greeting'), (['good', 'morning'], 'greeting'), (['good', 'afternoon'], 'greeting'), (['python'], 'python'), (['Tell', 'me', 'about', 'Python'], 'python'), (['What', 'can', 'you', 'tell', 'me', 'about', 'Python', '?'], 'python'), (['I', 'want', 'to', 'learn', 'Python'], 'python'), (['java'], 'java'), (['Can', 'you', 'tell', 'me', 'about', 'Java', '?'], 'java'), (['What', "'s", 'special', 'about', 'Java', '?'], 'java'), (['I', 'need', 'information', 'about', 'Java'], 'java'), (['javascript'], 'javascript'), (['What', 'can', 'you', 'tell', 'me', 'about', 'JavaScript', '?'], 'javascript'), (['Tell', 'me', 'more', 'about', 'JavaScript'], 'javascript'), (['I', 'want', 'to', 'learn', 'JavaScript'], 'javascript'), (['c++'], 'c++'), (['What', 'is', 'C++', '?'], 'c++'), (['Tell', 'me', 'about', 'C++'], 'c++'), (['Why', 'should', 'I', 'learn', 'C++', '?'], 'c++'), (['ruby'], 'ruby'), (['What', 'i

In [ ]:
pattern_words = ['Hello','is','sun','having']
pattern_words

['Hello', 'is', 'sun', 'having']

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words to lower case and lemmatize
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array, if word match found in current pattern then put 1 otherwise 0.
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # in output array 0 value for each tag and 1 value for matched tag.
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])  # Append the bag of words and output array as a list

# shuffle training
random.shuffle(training)

# Extract input and output data
train_x = np.array([item[0] for item in training])  # Extract bag of words
train_y = np.array([item[1] for item in training])  # Extract output arrays

print("Training data created")


Training data created


In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.02640143  0.00248569 -0.15232633 ...  0.01943405 -0.1582182
   0.10347129]
 [-0.07114959  0.16097392  0.0883031  ... -0.15224299 -0.11245032
  -0.10525242]
 [-0.16807246 -0.08150359  0.1552413  ... -0.15968907  0.14650829
  -0.00379887]
 ...
 [-0.15796879  0.08035998 -0.0636023  ... -0.01709767 -0.13074885
   0.17908244]
 [ 0.13959755 -0.04650787 -0.03828771 ... -0.04521118  0.11306737
  -0.15526599]
 [ 0.17107336  0.0966628   0.12902503 ... -0.00570969  0.17919503
  -0.12881418]]


In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
def clean_up_sentence(sentence):
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence)
    # Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    # Remove empty strings and convert to lowercase
    tokens = [word.lower() for word in tokens if word]
    return tokens

In [ ]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
14/14 [==============================] - 2s 6ms/step - loss: 2.8789 - accuracy: 0.0429
Epoch 2/200
14/14 [==============================] - 0s 3ms/step - loss: 2.8520 - accuracy: 0.0714
Epoch 3/200
14/14 [==============================] - 0s 4ms/step - loss: 2.8344 - accuracy: 0.0571
Epoch 4/200
14/14 [==============================] - 0s 3ms/step - loss: 2.7733 - accuracy: 0.1143
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 2.7686 - accuracy: 0.1143
Epoch 6/200
14/14 [==============================] - 0s 3ms/step - loss: 2.7381 - accuracy: 0.1714
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 2.6923 - accuracy: 0.1714
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 2.6392 - accuracy: 0.1571
Epoch 9/200
14/14 [==============================] - 0s 4ms/step - loss: 2.6020 - accuracy: 0.2714
Epoch 10/200
14/14 [==============================] - 0s 2ms/step - loss: 2.5415 - accuracy: 0.2571
Epoch 11/

In [ ]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    # Initialize bag of words
    bag = [0]*50  # Fixed length of 50
    for s in sentence_words:
        if s in words:
            # Get the index of the word in the vocabulary
            idx = words.index(s)
            if idx < 50:
                # Assign 1 if the index is within range
                bag[idx] = 1
            else:
                if show_details:
                    print("Word index out of range: %s" % s)
    return np.array(bag)


In [ ]:
import pickle

# Load words from file
words = pickle.load(open('words.pkl', 'rb'))

In [ ]:
bow("What can you tell me about Python?",words, show_details=True)

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1])

In [ ]:
words

["'s",
 'a',
 'about',
 'afternoon',
 'back',
 'be',
 'c++',
 'can',
 'cs',
 'developer',
 'development',
 'do',
 'doe',
 'end',
 'front',
 'full',
 'go',
 'good',
 'hello',
 'hey',
 'hi',
 'html',
 'i',
 'information',
 'is',
 'java',
 'javascript',
 'learn',
 'me',
 'mobile',
 'more',
 'morning',
 'need',
 'php',
 'python',
 'ruby',
 'should',
 'skill',
 'special',
 'sql',
 'stack',
 'swift',
 'tell',
 'to',
 'typescript',
 'want',
 'what',
 'why',
 'yo',
 'you']

In [ ]:
def predict_class(sentence, words):
    # Generate probabilities from the model
    p = bow(sentence, words, show_details=True)
    res = model.predict(np.array([p]))[0]  # Remove [50]
    # Get index of the maximum probability
    max_prob_index = np.argmax(res)
    # Get the corresponding class and probability
    predicted_class = classes[max_prob_index]
    confidence = res[max_prob_index]
    return [(predicted_class, confidence)]



In [ ]:
def get_response(ints):
    tag = ints[0][0]
    for intent in intents:
        if intent['tag'] == tag:
            responses = intent['responses']
            return random.choice(responses)
    return "I'm sorry, I don't understand that."

# Function to interact with the chatbot
def chatbot_response(sentence):
    ints = predict_class(sentence, words)  # Pass the 'words' argument here
    response = get_response(ints)
    return response

# Example usage:
words = ["'s", 'a', 'about', 'afternoon', 'back', 'be', 'c++', 'can', 'cs', 'developer', 'development', 'do', 'doe', 'end', 'front', 'full', 'go', 'good', 'hello', 'hey', 'hi', 'html', 'i', 'information', 'is', 'java', 'javascript', 'learn', 'me', 'mobile', 'more', 'morning', 'need', 'php', 'python', 'ruby', 'should', 'skill', 'special', 'sql', 'stack', 'swift', 'tell', 'to', 'typescript', 'want', 'what', 'why', 'yo', 'you']  # Define your words list here
print(chatbot_response("end"))


1/1 [==============================] - 0s 39ms/step
Front end developers focus on the user interface and user experience.
